## RWC-AI-Studio | The AI Art Quest

---

**The ultimate gamified AI image generation platform that transforms art creation into an epic RPG adventure!**

## 🏆 Game Features:
- 🎮 **RPG Progression System** - Level 1-50 with XP rewards
- 🎨 **Tiered Art Styles** - Common → Uncommon → Rare → Epic → Legendary
- 🏅 **Achievement System** - 6+ achievements with bonus XP
- ⚔️ **Quest-Based Generation** - Text Quests & Transform Quests
- 💎 **Player Profile** - Track your artistic journey and statistics

---

**🚀 Ready to begin your Art Quest adventure? Run the cells below!**

## ⚠️ IMPORTANT: Set Your Hugging Face Token!

Before running the next cell, you need to set your Hugging Face token:

1. 🔗 **Get your token**: Visit [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2. 🔑 **Create a token** with "Read" permissions
3. 📝 **Set it below** by replacing `"your_hf_token_here"` with your actual token

**Alternative**: Use Colab Secrets (Recommended)
- Go to the 🔑 key icon in Colab sidebar
- Add a secret named `HF_TOKEN` with your token value
- The code will automatically use it

In [1]:
# 🎮 RWC-AI-Studio | The AI Art Quest  - Dependency Installation
!pip install --quiet diffusers transformers accelerate safetensors xformers torch torchvision pillow flask flask-cors pydrive2 requests python-dotenv cryptography pyngrok

print("All dependencies installed successfully!")

All dependencies installed successfully!


In [2]:
import os
import json
import time
import base64
import traceback
from io import BytesIO
from typing import List, Dict, Any, Optional
from functools import wraps
from threading import Semaphore
import uuid
import datetime
import secrets
import hashlib

try:
    from google.colab import auth as colab_auth
except ImportError:
    colab_auth = None

from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
from werkzeug.utils import secure_filename
from PIL import Image
import torch
import requests

from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionPipeline
from transformers import logging as transformers_logging

try:
    from transformers import CLIPTextModel, CLIPTokenizer
except Exception:
    CLIPTextModel = None
    CLIPTokenizer = None

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

#  CONFIGURATION
OUTPUT_DIR = os.environ.get("OUTPUT_DIR", "/content/drive/MyDrive/ArtQuest/Gallery")
INPUT_DIR = os.environ.get("INPUT_DIR", "/content/drive/MyDrive/ArtQuest/Uploads")
META_DIR = os.environ.get("META_DIR", "/content/drive/MyDrive/ArtQuest/Progress")
PLAYER_DATA_DIR = os.environ.get("PLAYER_DATA_DIR", "/content/drive/MyDrive/ArtQuest/Players")

# Create game directories
for directory in [OUTPUT_DIR, INPUT_DIR, META_DIR, PLAYER_DATA_DIR]:
    os.makedirs(directory, exist_ok=True)

# Secrets Environment
MODEL_ID = os.environ.get("MODEL_ID", "runwayml/stable-diffusion-v1-5")
HF_TOKEN = os.environ.get("HF_TOKEN", None)  # ⚠️ SET YOUR TOKEN!
API_KEY = os.environ.get("API_KEY", f"artquest_{secrets.token_urlsafe(24)}")  # 🔐 Secure random key
DEVICE = os.environ.get("DEVICE", "cuda" if torch.cuda.is_available() else "cpu")

# OPTIMIZATIONS
ALLOWED_EXT = {'.png', '.jpg', '.jpeg', '.webp', '.bmp', '.tiff'}
MAX_UPLOAD_BYTES = int(os.environ.get("MAX_UPLOAD_BYTES", 10 * 1024 * 1024))  # 10MB
MAX_CONCURRENT_INFERENCES = int(os.environ.get("MAX_CONCURRENT", 2))  # Improved concurrency
inference_semaphore = Semaphore(MAX_CONCURRENT_INFERENCES)

print("🎮 ========== RWC-AI-Game: The AI Art Quest ========== 🎮")
print(f"🎯 Device: {DEVICE}")
print(f"🏛️ Gallery: {OUTPUT_DIR}")
print("🚀 Initializing gaming systems...")

🎮 ========== RWC-AI-Game: The AI Art Quest ========== 🎮
🎯 Device: cuda
🏛️ Gallery: /content/drive/MyDrive/ArtQuest/Gallery
🚀 Initializing gaming systems...


In [3]:
GAMIFIED_STYLE_PRESETS = {
    # 🌟 LEGENDARY TIER
    "legendary-fusion": {
        "add": "legendary masterpiece, divine artistry, transcendent beauty, perfect harmony",
        "neg": "amateur, simple, basic, low quality",
        "tier": "legendary",
        "unlock_level": 50,
        "xp_bonus": 500,
        "description": "The ultimate fusion of all artistic mastery"
    },
    "quantum-dreams": {
        "add": "quantum realm visualization, interdimensional art, reality-bending aesthetics",
        "neg": "normal physics, mundane reality, simple concepts",
        "tier": "legendary",
        "unlock_level": 45,
        "xp_bonus": 400,
        "description": "Art that transcends dimensional boundaries"
    },

    # 🔥 EPIC TIER
    "cyberpunk-noir": {
        "add": "cyberpunk noir, neon-soaked streets, dark future, rain-slicked chrome",
        "neg": "bright, cheerful, pastoral, vintage",
        "tier": "epic",
        "unlock_level": 30,
        "xp_bonus": 200,
        "description": "Dark future meets classic detective"
    },
    "dragon-forge": {
        "add": "dragon's forge, molten metal, mythical craftsmanship, fire and steel",
        "neg": "cold, sterile, modern, simple",
        "tier": "epic",
        "unlock_level": 25,
        "xp_bonus": 180,
        "description": "Forged in the flames of ancient dragons"
    },
    "time-weaver": {
        "add": "temporal distortion, time streams, chronological layers, past-future blend",
        "neg": "static time, single moment, simple timeline",
        "tier": "epic",
        "unlock_level": 35,
        "xp_bonus": 220,
        "description": "Weaving threads through time itself"
    },

    # 💎 RARE TIER
    "steampunk-fantasy": {
        "add": "steampunk fantasy, brass clockwork, Victorian magic, mechanical dragons",
        "neg": "modern, digital, simple, clean",
        "tier": "rare",
        "unlock_level": 15,
        "xp_bonus": 100,
        "description": "Victorian engineering meets magical realms"
    },
    "neon-samurai": {
        "add": "neon samurai, cybernetic katana, honor in digital age, warrior's code",
        "neg": "dishonor, modern weapons, casual, weak",
        "tier": "rare",
        "unlock_level": 20,
        "xp_bonus": 120,
        "description": "Ancient honor in a neon world"
    },
    "crystal-caverns": {
        "add": "crystal caverns, prismatic light, gem formations, underground palace",
        "neg": "surface, dull, matte, simple caves",
        "tier": "rare",
        "unlock_level": 18,
        "xp_bonus": 110,
        "description": "Deep earth's crystalline treasures"
    },
    "void-walker": {
        "add": "void walker, space wanderer, cosmic horror, interdimensional travel",
        "neg": "earthbound, safe, familiar, comfortable",
        "tier": "rare",
        "unlock_level": 22,
        "xp_bonus": 130,
        "description": "Wandering through cosmic emptiness"
    },

    # ⭐ UNCOMMON TIER
    "pixel-knight": {
        "add": "pixel art knight, 8-bit armor, retro gaming aesthetic, blocky heroism",
        "neg": "smooth, high-res, modern graphics, realistic",
        "tier": "uncommon",
        "unlock_level": 8,
        "xp_bonus": 60,
        "description": "Retro gaming meets medieval valor"
    },
    "magic-realism": {
        "add": "magical realism, subtle enchantment, everyday magic, wonder in mundane",
        "neg": "purely realistic, no magic, completely fantastical",
        "tier": "uncommon",
        "unlock_level": 10,
        "xp_bonus": 70,
        "description": "Magic hidden in plain sight"
    },
    "art-nouveau-cyber": {
        "add": "art nouveau cyberpunk, organic circuits, flowing tech, natural cyber",
        "neg": "harsh angles, mechanical, sterile, basic tech",
        "tier": "uncommon",
        "unlock_level": 12,
        "xp_bonus": 80,
        "description": "Organic beauty meets digital future"
    },
    "ghost-in-machine": {
        "add": "ghost in the shell, digital souls, AI consciousness, electronic spirits",
        "neg": "purely physical, no AI, simple technology, soulless",
        "tier": "uncommon",
        "unlock_level": 14,
        "xp_bonus": 90,
        "description": "Where consciousness meets code"
    },

    # 🔰 COMMON TIER (Always available)
    "fantasy": {
        "add": "epic fantasy, magical realms, heroic adventure, mystical creatures",
        "neg": "modern, mundane, realistic, ordinary",
        "tier": "common",
        "unlock_level": 1,
        "xp_bonus": 25,
        "description": "Classic magical adventures"
    },
    "cyberpunk": {
        "add": "cyberpunk cityscape, neon lights, high-tech low-life, urban future",
        "neg": "natural, rural, past, low-tech",
        "tier": "common",
        "unlock_level": 1,
        "xp_bonus": 30,
        "description": "High-tech dystopian futures"
    },
    "anime": {
        "add": "anime style, manga aesthetics, expressive characters, vibrant colors",
        "neg": "western art, realistic proportions, muted colors",
        "tier": "common",
        "unlock_level": 1,
        "xp_bonus": 25,
        "description": "Japanese animation artistry"
    },
    "photorealistic": {
        "add": "photorealistic, ultra detailed, professional photography, perfect clarity",
        "neg": "stylized, artistic, blurry, low detail",
        "tier": "common",
        "unlock_level": 1,
        "xp_bonus": 40,
        "description": "Camera-perfect realism"
    },
    "watercolor": {
        "add": "watercolor painting, soft washes, flowing pigments, delicate beauty",
        "neg": "harsh lines, digital, bold, aggressive",
        "tier": "common",
        "unlock_level": 1,
        "xp_bonus": 30,
        "description": "Gentle flowing paint techniques"
    },
    "pixel-art": {
        "add": "pixel art, 8-bit retro, blocky style, gaming nostalgia",
        "neg": "smooth, high-resolution, modern, realistic",
        "tier": "common",
        "unlock_level": 1,
        "xp_bonus": 35,
        "description": "Retro gaming aesthetics"
    },
    "comic-book": {
        "add": "comic book style, bold lines, dynamic action, superhero aesthetics",
        "neg": "subtle, realistic, muted, static",
        "tier": "common",
        "unlock_level": 1,
        "xp_bonus": 30,
        "description": "Dynamic comic book action"
    }
}

print(f"🎨 Loaded {len(GAMIFIED_STYLE_PRESETS)} gamified art styles!")

🎨 Loaded 20 gamified art styles!


In [4]:
class GameSystem:
    @staticmethod
    def calculate_level(total_xp):
        # Calculate player level based on total XP
        if total_xp < 100:
            return 1
        return min(50, int((total_xp / 100) ** 0.5) + 1)

    @staticmethod
    def xp_for_next_level(current_level):
        #Calculate XP needed for next level
        return ((current_level) ** 2) * 100

    @staticmethod
    def calculate_generation_xp(style_tier, steps, quality_bonus=0):
        # Calculate XP earned from generation
        base_xp = {
            "common": 10,
            "uncommon": 25,
            "rare": 50,
            "epic": 100,
            "legendary": 200
        }.get(style_tier, 10)

        step_bonus = min(steps // 10, 20)  # Max 20 bonus for steps
        return base_xp + step_bonus + quality_bonus

# 🏆 ACHIEVEMENT SYSTEM
ACHIEVEMENTS = {
    "first_quest": {
        "name": "First Quest",
        "description": "Generate your first AI artwork",
        "xp_reward": 50,
        "condition": lambda stats: stats.get("total_generations", 0) >= 1
    },
    "style_explorer": {
        "name": "Style Explorer",
        "description": "Try 5 different art styles",
        "xp_reward": 100,
        "condition": lambda stats: len(stats.get("styles_used", [])) >= 5
    },
    "epic_collector": {
        "name": "Epic Collector",
        "description": "Unlock your first Epic tier style",
        "xp_reward": 200,
        "condition": lambda stats: stats.get("player_level", 1) >= 25
    },
    "legendary_artist": {
        "name": "Legendary Artist",
        "description": "Reach level 50 and unlock Legendary styles",
        "xp_reward": 1000,
        "condition": lambda stats: stats.get("player_level", 1) >= 50
    },
    "speed_demon": {
        "name": "Speed Demon",
        "description": "Generate 10 images in one session",
        "xp_reward": 150,
        "condition": lambda stats: stats.get("session_generations", 0) >= 10
    },
    "master_crafter": {
        "name": "Master Crafter",
        "description": "Generate 100 total artworks",
        "xp_reward": 500,
        "condition": lambda stats: stats.get("total_generations", 0) >= 100
    }
}

print(f"🎯 Start generating art to earn XP and level up! Loaded {len(ACHIEVEMENTS)} achievements to unlock!")

🎯 Start generating art to earn XP and level up! Loaded 6 achievements to unlock!


In [5]:
# 📊 PLAYER DATA MANAGEMENT
def get_player_data(player_id=None):
    # Load || create player progress data
    if not player_id:
        player_id = "tester on colab"

    player_file = os.path.join(PLAYER_DATA_DIR, f"{hashlib.md5(player_id.encode()).hexdigest()}.json")

    if os.path.exists(player_file):
        try:
            with open(player_file, 'r') as f:
                return json.load(f)
        except:
            pass

    # Create new player
    return {
        "player_id": player_id,
        "total_xp": 0,
        "player_level": 1,
        "total_generations": 0,
        "session_generations": 0,
        "styles_used": [],
        "achievements_unlocked": [],
        "favorite_styles": [],
        "creation_streak": 0,
        "legendary_unlocks": 0,
        "join_date": datetime.datetime.now().isoformat()
    }

def save_player_data(player_data):
    # Save player progress data
    player_id = player_data.get("player_id", "unknown")
    player_file = os.path.join(PLAYER_DATA_DIR, f"{hashlib.md5(player_id.encode()).hexdigest()}.json")

    try:
        with open(player_file, 'w') as f:
            json.dump(player_data, f, indent=2)
    except Exception as e:
        print(f"⚠️ Failed to save player data: {e}")

def update_player_progress(player_data, style_tier, steps):
    # Update player progress after generation and Calculate XP earned
    xp_earned = GameSystem.calculate_generation_xp(style_tier, steps)
    player_data["total_xp"] += xp_earned
    player_data["total_generations"] += 1
    player_data["session_generations"] += 1

    # Update level
    old_level = player_data["player_level"]
    new_level = GameSystem.calculate_level(player_data["total_xp"])
    player_data["player_level"] = new_level

    # Check for achievements
    new_achievements = []
    for ach_id, achievement in ACHIEVEMENTS.items():
        if ach_id not in player_data["achievements_unlocked"]:
            if achievement["condition"](player_data):
                player_data["achievements_unlocked"].append(ach_id)
                player_data["total_xp"] += achievement["xp_reward"]
                new_achievements.append(achievement)

    return {
        "xp_earned": xp_earned,
        "level_up": new_level > old_level,
        "new_level": new_level,
        "new_achievements": new_achievements,
        "total_xp": player_data["total_xp"]
    }

# Initialize or load player data
CURRENT_PLAYER = get_player_data()
print(f"👤 Welcome back! You are level {CURRENT_PLAYER['player_level']} && You have created {CURRENT_PLAYER['total_generations']} artworks, As well as 🏆 Achievements unlocked: {len(CURRENT_PLAYER['achievements_unlocked'])}")

👤 Welcome back! You are level 2 && You have created 4 artworks, As well as 🏆 Achievements unlocked: 1


In [ ]:
# FLASK SETUP
app = Flask(__name__)
CORS(app)
app.app_context().push()

print("🎮 Initializing RWC-AI-Studio | The AI Art Quest Gaming Platform...")

# GOOGLE DRIVE INTEGRATION
gdrive = None
try:
    if colab_auth is not None:
        # print("🔐 Authenticating Google Colab user...")
        colab_auth.authenticate_user()
        # print("✅ Google Colab user authenticated.")

        gauth = GoogleAuth('Colab')
        gdrive = GoogleDrive(gauth)
        # print("☁️ GoogleDrive client initialized successfully.")
    else:
        print("⚠️ google.colab.auth not available, GoogleDrive will not be initialized.")
except Exception as e:
    print(f"❌ GoogleDrive initialization failed: {e}")
    gdrive = None

# MODEL LOADING
pipe = None
text_pipe = None
use_fp16 = (DEVICE != 'cpu') and torch.cuda.is_available()
torch_dtype = torch.float16 if use_fp16 else torch.float32

print(f"🤖 Loading AI Models: {MODEL_ID} on {DEVICE} (FP16: {use_fp16})")

In [ ]:
# 🔑 SET YOUR HUGGING FACE TOKEN HERE
import os
try:
    from google.colab import userdata
    # Try to get token from Colab secrets first
    try:
        HF_TOKEN = userdata.get('HF_TOKEN')
        print("✅ Using HF_TOKEN from Colab secrets")
    except userdata.SecretNotFoundError:
        # Fallback: Set your token manually here
        HF_TOKEN = None

        if HF_TOKEN == "your_hf_token_here" or not HF_TOKEN:
            print("❌ Please set your Hugging Face token!")
        else:
            print(f"✅ Using manually set HF_TOKEN.")
except ImportError:
    # Not in Colab environment
    HF_TOKEN = os.environ.get('HF_TOKEN', 'your_hf_token_here')
    if HF_TOKEN == 'your_hf_token_here' or not HF_TOKEN:
        print("❌ Please set HF_TOKEN environment variable!")
    else:
        print(f"✅ Using HF_TOKEN from environment.")

# Update the environment variable
os.environ['HF_TOKEN'] = HF_TOKEN

print("🎮 Token configuration complete!")

In [8]:
# 🤖 LOAD AI MODELS
try:
    # print("🎨 Loading Img2Img pipeline...")
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        MODEL_ID, torch_dtype=torch_dtype, token=HF_TOKEN
    )
    pipe = pipe.to(DEVICE)
    if hasattr(pipe, "enable_xformers_memory_efficient_attention"):
        pipe.enable_xformers_memory_efficient_attention()
    print("✅ Img2Img pipeline loaded.")
except Exception as e:
    print(f"⚠️ Img2Img load failed: {e}")
    pipe = None

try:
    # print("📝 Loading Text2Img pipeline...")
    text_pipe = StableDiffusionPipeline.from_pretrained(
        MODEL_ID, torch_dtype=torch_dtype, token=HF_TOKEN
    )
    text_pipe = text_pipe.to(DEVICE)
    if hasattr(text_pipe, "enable_xformers_memory_efficient_attention"):
        text_pipe.enable_xformers_memory_efficient_attention()
    print("✅ Text2Img pipeline loaded.")
except Exception as e:
    print(f"⚠️ Text2Img load failed: {e}")
    text_pipe = None

print("🎮 Art Quest Studio models ready!")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


✅ Img2Img pipeline loaded.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Text2Img pipeline loaded.
🎮 Art Quest Studio models ready!


In [8]:
# UTILITY FUNCTIONS
def gamified_filename(player_level, style_tier, batch_id, role, ext, idx):
    ts = datetime.datetime.now(datetime.timezone.utc).strftime('%Y%m%dT%H%M%S%f')[:-3]
    tier_prefix = style_tier[:3].upper()
    return f"LV{player_level}_{tier_prefix}_{batch_id}_{role}_{idx:04d}_{ts}{ext}"

def save_pil_with_metadata(img: Image.Image, path: str, metadata: dict):
    # Save image with embedded metadata
    os.makedirs(os.path.dirname(path), exist_ok=True)

    from PIL.PngImagePlugin import PngInfo
    png_info = PngInfo()
    for key, value in metadata.items():
        png_info.add_text(key, str(value))

    img.save(path, pnginfo=png_info, quality=95, optimize=True)

def pil_to_base64(img: Image.Image, fmt='PNG') -> str:
    # Convert PIL image to base64
    buf = BytesIO()
    img.save(buf, format=fmt, quality=95, optimize=True)
    buf.seek(0)
    return f"data:image/{fmt.lower()};base64,{base64.b64encode(buf.read()).decode('ascii')}"

def safe_int(value, default=0):
    # convert to integer
    try:
        return int(value)
    except (ValueError, TypeError):
        return default

def safe_float(value, default=0.0):
    # convert to float
    try:
        return float(value)
    except (ValueError, TypeError):
        return default

print("🛠️ Utility functions loaded!")

🛠️ Utility functions loaded!


In [9]:
# 🎮 GENERATION FUNCTION
def generate_art_quest(prompt, style_id="fantasy", negative_prompt="", steps=35, guidance=7.5, mode="text", input_image=None, strength=0.7):

    global CURRENT_PLAYER

    if style_id not in GAMIFIED_STYLE_PRESETS:
        style_id = "fantasy"  # Default fallback

    style_data = GAMIFIED_STYLE_PRESETS[style_id]

    if CURRENT_PLAYER["player_level"] < style_data["unlock_level"]:
        return {
            "success": False,
            "error": f"🔒 Style '{style_id}' requires level {style_data['unlock_level']}! You are level {CURRENT_PLAYER['player_level']}"
        }

    # Apply style
    enhanced_prompt = f"{prompt}, {style_data['add']}"
    if not negative_prompt:
        negative_prompt = style_data.get('neg', '')

    # Add style to used styles
    if style_id not in CURRENT_PLAYER["styles_used"]:
        CURRENT_PLAYER["styles_used"].append(style_id)

    # Generate artwork
    print(f"🎨 Starting {mode} quest with {style_data['tier']} tier style...")
    print(f"⚡ Using {steps} steps for quality generation")

    try:
        if mode == "text" and text_pipe is not None:
            result_image = text_pipe(
                prompt=enhanced_prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=steps,
                guidance_scale=guidance
            ).images[0]
        elif mode == "image" and pipe is not None and input_image is not None:
            result_image = pipe(
                prompt=enhanced_prompt,
                image=input_image,
                negative_prompt=negative_prompt,
                strength=strength,
                guidance_scale=guidance,
                num_inference_steps=steps
            ).images[0]
        else:
            return {
                "success": False,
                "error": "❌ Required pipeline not loaded or invalid mode"
            }

        # Save artwork with metadata
        batch_id = f"quest_{uuid.uuid4().hex[:8]}"
        filename = gamified_filename(
            CURRENT_PLAYER["player_level"], style_data["tier"], batch_id, "artwork", ".png", 0
        )
        output_path = os.path.join(OUTPUT_DIR, filename)

        metadata = {
            "player_level": CURRENT_PLAYER["player_level"],
            "style_tier": style_data["tier"],
            "style_id": style_id,
            "prompt": prompt,
            "enhanced_prompt": enhanced_prompt,
            "steps": steps,
            "guidance": guidance,
            "mode": mode,
            "creation_date": datetime.datetime.now().isoformat()
        }

        if mode == "image":
            metadata["strength"] = strength

        save_pil_with_metadata(result_image, output_path, metadata)

        # Update player progress
        progress_update = update_player_progress(CURRENT_PLAYER, style_data["tier"], steps)
        save_player_data(CURRENT_PLAYER)

        print(f"🏆 Quest complete! Earned {progress_update['xp_earned']} XP")
        if progress_update['level_up']:
            print(f"🎉 LEVEL UP! You are now level {progress_update['new_level']}!")

        if progress_update['new_achievements']:
            for achievement in progress_update['new_achievements']:
                print(f"🏅 Achievement Unlocked: {achievement['name']} (+{achievement['xp_reward']} XP)")

        return {
            "success": True,
            "image": result_image,
            "base64": pil_to_base64(result_image),
            "filename": filename,
            "output_path": output_path,
            "style_applied": style_id,
            "style_tier": style_data["tier"],
            "metadata": metadata,
            "player_progress": progress_update
        }

    except Exception as e:
        print(f"❌ Quest failed: {str(e)}")
        traceback.print_exc()
        return {
            "success": False,
            "error": f"Generation failed: {str(e)}"
        }

print("🎮 RWC-AI-Studio | The AI Art Quest generation system ready! 🚀 Use generate_art_quest() to create your masterpieces!")

🎮 RWC-AI-Studio | The AI Art Quest generation system ready! 🚀 Use generate_art_quest() to create your masterpieces!


In [ ]:
# FULL WEB INTERFACE WITH NGROK
import threading
from pyngrok import ngrok

# API Authentication
def require_api_key(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        key = request.headers.get("X-API-KEY") or request.args.get("api_key")
        if not key or key != API_KEY:
            return jsonify({"error": "unauthorized", "message": "Invalid API key"}), 401
        return f(*args, **kwargs)
    return wrapper

# API ROUTES
@app.route('/api/player/profile', methods=['GET'])
def get_player_profile():
    unlocked_styles = []
    for style_id, style_data in GAMIFIED_STYLE_PRESETS.items():
        if CURRENT_PLAYER["player_level"] >= style_data["unlock_level"]:
            unlocked_styles.append({
                "id": style_id,
                "name": style_id.replace("-", " ").title(),
                "tier": style_data["tier"],
                "description": style_data["description"],
                "xp_bonus": style_data["xp_bonus"]
            })

    return jsonify({
        "player_level": CURRENT_PLAYER["player_level"],
        "total_xp": CURRENT_PLAYER["total_xp"],
        "xp_for_next_level": GameSystem.xp_for_next_level(CURRENT_PLAYER["player_level"]),
        "total_generations": CURRENT_PLAYER["total_generations"],
        "achievements_unlocked": len(CURRENT_PLAYER["achievements_unlocked"]),
        "unlocked_styles": unlocked_styles,
        "achievements": [
            {
                "id": ach_id,
                "name": ACHIEVEMENTS[ach_id]["name"],
                "description": ACHIEVEMENTS[ach_id]["description"],
                "unlocked": ach_id in CURRENT_PLAYER["achievements_unlocked"]
            }
            for ach_id in ACHIEVEMENTS.keys()
        ]
    })

@app.route('/api/styles', methods=['GET'])
def get_gamified_styles():
    player_level = CURRENT_PLAYER["player_level"]

    styles_by_tier = {"common": [], "uncommon": [], "rare": [], "epic": [], "legendary": []}

    for style_id, style_data in GAMIFIED_STYLE_PRESETS.items():
        tier = style_data["tier"]
        is_unlocked = player_level >= style_data["unlock_level"]

        style_info = {
            "id": style_id,
            "name": style_id.replace("-", " ").title(),
            "tier": tier,
            "description": style_data["description"],
            "unlock_level": style_data["unlock_level"],
            "xp_bonus": style_data["xp_bonus"],
            "unlocked": is_unlocked
        }

        styles_by_tier[tier].append(style_info)

    return jsonify({
        "styles_by_tier": styles_by_tier,
        "player_level": player_level,
        "total_styles": len(GAMIFIED_STYLE_PRESETS)
    })

@app.route('/api/generate_text', methods=['POST'])
@require_api_key
def api_generate_text():
    try:
        data = request.get_json(force=True)
    except Exception:
        return jsonify({"error": "invalid_json"}), 400

    prompt = (data.get("prompt") or "").strip()
    if not prompt:
        return jsonify({"error": "missing_prompt"}), 400

    result = generate_art_quest(
        prompt=prompt,
        style_id=data.get("style", "fantasy"),
        negative_prompt=data.get("negative_prompt", ""),
        steps=safe_int(data.get("num_inference_steps", 35), 35),
        guidance=safe_float(data.get("guidance_scale", 7.5), 7.5),
        mode="text"
    )

    if result["success"]:
        return jsonify({
            "success": True,
            "base64": result["base64"],
            "style_applied": result["style_applied"],
            "style_tier": result["style_tier"],
            "player_progress": result["player_progress"],
            "metadata": result["metadata"]
        })
    else:
        return jsonify({"success": False, "error": result["error"]}), 400

@app.route('/api/generate_from_upload', methods=['POST'])
@require_api_key
def api_generate_from_upload():
    file = request.files.get('file')
    if not file:
        return jsonify({"error": "no_file"}), 400

    try:
        input_image = Image.open(file.stream).convert("RGB")
    except Exception:
        return jsonify({"error": "invalid_image"}), 400

    prompt = (request.form.get("prompt") or "").strip()

    result = generate_art_quest(
        prompt=prompt,
        style_id=request.form.get("style", "fantasy"),
        negative_prompt=request.form.get("negative_prompt", ""),
        steps=safe_int(request.form.get("num_inference_steps", 25), 25),
        guidance=safe_float(request.form.get("guidance_scale", 8.0), 8.0),
        strength=safe_float(request.form.get("strength", 0.7), 0.7),
        mode="image",
        input_image=input_image
    )

    if result["success"]:
        return jsonify({
            "success": True,
            "base64": result["base64"],
            "style_applied": result["style_applied"],
            "style_tier": result["style_tier"],
            "player_progress": result["player_progress"],
            "metadata": result["metadata"]
        })
    else:
        return jsonify({"success": False, "error": result["error"]}), 400

print("🌐 API routes configured!")

In [ ]:
# 🎮 RWC-AI-Studio | The AI Art Quest - Gamified User interface
GAMIFIED_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>RWC-AI-Studio | The AI Art Quest </title>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Orbitron:wght@400;700;900&family=Exo+2:wght@300;400;600;700;800&display=swap');

        :root {
            --primary-gold: #FFD700;
            --legendary-purple: #9D4EDD;
            --epic-orange: #FF6B35;
            --rare-blue: #4ECDC4;
            --uncommon-green: #45B7D1;
            --common-gray: #95A5A6;

            --bg-dark: #0D1117;
            --bg-card: #161B22;
            --bg-accent: #21262D;
            --text-primary: #F0F6FC;
            --text-secondary: #8B949E;
            --border: #30363D;
            --success: #2EA043;
            --warning: #FB8500;
            --error: #F85149;
        }

        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: 'Exo 2', sans-serif;
            background: linear-gradient(135deg, var(--bg-dark) 0%, #1C2128 100%);
            color: var(--text-primary);
            min-height: 100vh;
        }

        .container {
            max-width: 1400px;
            margin: 0 auto;
            padding: 20px;
        }

        .header {
            text-align: center;
            margin-bottom: 30px;
        }

        .header h1 {
            font-family: 'Orbitron', monospace;
            font-size: 3.5rem;
            font-weight: 900;
            background: linear-gradient(45deg, var(--primary-gold), var(--legendary-purple), var(--epic-orange));
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
            margin-bottom: 10px;
        }

        .player-hud {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }

        .hud-card {
            background: var(--bg-card);
            border: 2px solid var(--border);
            border-radius: 15px;
            padding: 20px;
            text-align: center;
        }

        .game-panel {
            background: var(--bg-card);
            border: 2px solid var(--border);
            border-radius: 20px;
            padding: 30px;
            margin-bottom: 30px;
        }

        .form-grid {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 30px;
        }

        .form-group {
            margin-bottom: 20px;
        }

        .form-label {
            display: block;
            margin-bottom: 8px;
            font-weight: 600;
            color: var(--primary-gold);
            text-transform: uppercase;
        }

        .form-input {
            width: 100%;
            padding: 15px;
            background: var(--bg-accent);
            border: 2px solid var(--border);
            border-radius: 8px;
            color: var(--text-primary);
            font-size: 16px;
        }

        .style-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(180px, 1fr));
            gap: 10px;
            max-height: 300px;
            overflow-y: auto;
            padding: 10px;
            background: var(--bg-accent);
            border-radius: 8px;
        }

        .style-card {
            padding: 12px;
            background: var(--bg-dark);
            border: 2px solid var(--border);
            border-radius: 8px;
            cursor: pointer;
            text-align: center;
            transition: all 0.3s;
        }

        .style-card:hover {
            transform: translateY(-2px);
        }

        .style-card.selected {
            border-color: var(--primary-gold);
            box-shadow: 0 0 20px rgba(255, 215, 0, 0.5);
        }

        .style-card.locked {
            opacity: 0.5;
            cursor: not-allowed;
        }

        .quest-btn {
            width: 100%;
            padding: 18px;
            background: linear-gradient(45deg, var(--success), var(--uncommon-green));
            border: none;
            border-radius: 12px;
            color: white;
            font-family: 'Orbitron', monospace;
            font-weight: 700;
            font-size: 1.1rem;
            cursor: pointer;
            transition: all 0.3s;
        }

        .quest-btn:hover {
            transform: translateY(-2px);
        }

        .quest-btn:disabled {
            opacity: 0.6;
            cursor: not-allowed;
        }

        .file-upload {
            position: relative;
            display: block;
            cursor: pointer;
        }

        .file-upload input {
            position: absolute;
            opacity: 0;
            width: 100%;
            height: 100%;
        }

        .file-upload-area {
            padding: 30px;
            background: var(--bg-accent);
            border: 2px dashed var(--border);
            border-radius: 8px;
            text-align: center;
        }

        .results-panel {
            margin-top: 30px;
            text-align: center;
        }

        .generated-image {
            max-width: 100%;
            border-radius: 12px;
            border: 3px solid var(--primary-gold);
        }

        .mode-selector {
            display: flex;
            gap: 10px;
            margin-bottom: 20px;
        }

        .mode-btn {
            flex: 1;
            padding: 15px;
            background: var(--bg-accent);
            border: 2px solid var(--border);
            border-radius: 8px;
            color: var(--text-primary);
            cursor: pointer;
            font-weight: 600;
        }

        .mode-btn.active {
            background: var(--primary-gold);
            color: var(--bg-dark);
        }

        .tier-filters {
            display: flex;
            gap: 8px;
            margin-bottom: 10px;
            flex-wrap: wrap;
        }

        .tier-filter {
            padding: 5px 10px;
            background: var(--bg-accent);
            border: 1px solid var(--border);
            border-radius: 15px;
            font-size: 0.8rem;
            cursor: pointer;
        }

        .tier-filter.active {
            background: var(--primary-gold);
            color: var(--bg-dark);
        }

        @media (max-width: 768px) {
            .form-grid {
                grid-template-columns: 1fr;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>RWC AI Studio</h1>
            <p>The AI Art Quest</p>
        </div>

        <!-- Player HUD -->
        <div class="player-hud">
            <div class="hud-card">
                <h3>🏆 Level</h3>
                <div style="font-size: 2rem; color: var(--legendary-purple);" id="playerLevel">1</div>
            </div>
            <div class="hud-card">
                <h3>⭐ Total XP</h3>
                <div style="font-size: 2rem; color: var(--primary-gold);" id="totalXP">0</div>
            </div>
            <div class="hud-card">
                <h3>🎨 Artworks</h3>
                <div style="font-size: 2rem; color: var(--rare-blue);" id="totalArt">0</div>
            </div>
            <div class="hud-card">
                <h3>🏅 Achievements</h3>
                <div style="font-size: 2rem; color: var(--epic-orange);" id="achievements">0</div>
            </div>
        </div>

        <!-- Game Panel -->
        <div class="game-panel">
            <div class="mode-selector">
                <button class="mode-btn active" onclick="setMode('text')" id="textMode">📝 Text Quest</button>
                <button class="mode-btn" onclick="setMode('image')" id="imageMode">🖼️ Transform Quest</button>
            </div>

            <div class="form-grid">
                <div>
                    <!-- Image Upload -->
                    <div class="form-group" id="imageUpload" style="display:none;">
                        <label class="form-label">Upload Image</label>
                        <div class="file-upload">
                            <input type="file" id="fileInput" accept="image/*" onchange="handleFileSelect()">
                            <div class="file-upload-area" id="uploadArea">
                                📤 Click to upload image<br>
                                <small>PNG, JPG, JPEG (max 10MB)</small>
                            </div>
                        </div>
                    </div>

                    <div class="form-group">
                        <label class="form-label">✨ Quest Description</label>
                        <textarea id="promptInput" class="form-input" rows="4" placeholder="Describe your vision..."></textarea>
                    </div>

                    <div class="form-group">
                        <label class="form-label">🚫 Negative Prompt</label>
                        <textarea id="negativePrompt" class="form-input" rows="3" placeholder="What to avoid..."></textarea>
                    </div>

                    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px;">
                        <div class="form-group">
                            <label class="form-label">Steps</label>
                            <input type="number" id="stepsInput" class="form-input" value="35" min="10" max="100">
                        </div>
                        <div class="form-group">
                            <label class="form-label">Guidance</label>
                            <input type="number" id="guidanceInput" class="form-input" value="7.5" min="1" max="20" step="0.5">
                        </div>
                    </div>

                    <div class="form-group" id="strengthGroup" style="display:none;">
                        <label class="form-label">Transform Strength</label>
                        <input type="number" id="strengthInput" class="form-input" value="0.7" min="0.1" max="1" step="0.1">
                    </div>
                </div>

                <div>
                    <div class="form-group">
                        <label class="form-label">🎨 Art Style</label>
                        <div class="tier-filters">
                            <button class="tier-filter active" onclick="filterStyles('all')">All</button>
                            <button class="tier-filter" onclick="filterStyles('common')">Common</button>
                            <button class="tier-filter" onclick="filterStyles('uncommon')">Uncommon</button>
                            <button class="tier-filter" onclick="filterStyles('rare')">Rare</button>
                            <button class="tier-filter" onclick="filterStyles('epic')">Epic</button>
                            <button class="tier-filter" onclick="filterStyles('legendary')">Legendary</button>
                        </div>
                        <div class="style-grid" id="styleGrid"></div>
                    </div>

                    <button class="quest-btn" onclick="startQuest()" id="questBtn">
                        🚀 Begin Art Quest
                    </button>
                </div>
            </div>
        </div>

        <!-- Results -->
        <div class="results-panel" id="resultsPanel" style="display:none;">
            <div id="statusMessage"></div>
            <div id="artworkResult"></div>
        </div>
    </div>

    <script>
        let currentMode = 'text';
        let selectedStyle = 'fantasy';
        let availableStyles = {};
        let currentFilter = 'all';
        let playerData = {};
        let API_KEY = '""" + API_KEY + """';

        // Initialize on load
        document.addEventListener('DOMContentLoaded', initializeGame);

        async function initializeGame() {
            await loadPlayerProfile();
            await loadStyles();
            updatePlayerHUD();
        }

        async function loadPlayerProfile() {
            try {
                const response = await fetch('/api/player/profile');
                playerData = await response.json();
            } catch (error) {
                console.error('Failed to load player profile:', error);
            }
        }

        async function loadStyles() {
            try {
                const response = await fetch('/api/styles');
                const data = await response.json();
                availableStyles = data.styles_by_tier;
                renderStyles();
            } catch (error) {
                console.error('Failed to load styles:', error);
            }
        }

        function updatePlayerHUD() {
            document.getElementById('playerLevel').textContent = playerData.player_level || 1;
            document.getElementById('totalXP').textContent = playerData.total_xp || 0;
            document.getElementById('totalArt').textContent = playerData.total_generations || 0;
            document.getElementById('achievements').textContent = playerData.achievements_unlocked || 0;
        }

        function renderStyles() {
            const styleGrid = document.getElementById('styleGrid');
            styleGrid.innerHTML = '';

            let stylesToShow = [];
            if (currentFilter === 'all') {
                for (const tier in availableStyles) {
                    stylesToShow = stylesToShow.concat(availableStyles[tier]);
                }
            } else {
                stylesToShow = availableStyles[currentFilter] || [];
            }

            stylesToShow.forEach(style => {
                const styleCard = document.createElement('div');
                styleCard.className = `style-card ${style.unlocked ? '' : 'locked'} ${selectedStyle === style.id ? 'selected' : ''}`;
                styleCard.onclick = () => selectStyle(style.id, style.unlocked);

                styleCard.innerHTML = `
                    <div style="font-weight: bold; margin-bottom: 5px;">${style.name}</div>
                    <div style="font-size: 0.8rem; color: var(--${style.tier === 'legendary' ? 'legendary-purple' : style.tier === 'epic' ? 'epic-orange' : style.tier === 'rare' ? 'rare-blue' : style.tier === 'uncommon' ? 'uncommon-green' : 'common-gray'});">${style.tier.toUpperCase()}</div>
                    <div style="font-size: 0.7rem; margin: 5px 0;">${style.description}</div>
                    ${style.unlocked ?
                        `<div style="font-size: 0.7rem; color: var(--primary-gold);">+${style.xp_bonus} XP</div>` :
                        `<div style="font-size: 0.7rem; color: var(--text-secondary);">Level ${style.unlock_level}</div>`
                    }
                `;

                styleGrid.appendChild(styleCard);
            });
        }

        function filterStyles(tier) {
            currentFilter = tier;
            document.querySelectorAll('.tier-filter').forEach(btn => btn.classList.remove('active'));
            event.target.classList.add('active');
            renderStyles();
        }

        function selectStyle(styleId, unlocked) {
            if (!unlocked) {
                alert('This style is locked! Level up to unlock it.');
                return;
            }
            selectedStyle = styleId;
            renderStyles();
        }

        function setMode(mode) {
            currentMode = mode;
            document.querySelectorAll('.mode-btn').forEach(btn => btn.classList.remove('active'));

            if (mode === 'text') {
                document.getElementById('textMode').classList.add('active');
                document.getElementById('imageUpload').style.display = 'none';
                document.getElementById('strengthGroup').style.display = 'none';
            } else {
                document.getElementById('imageMode').classList.add('active');
                document.getElementById('imageUpload').style.display = 'block';
                document.getElementById('strengthGroup').style.display = 'block';
            }
        }

        function handleFileSelect() {
            const fileInput = document.getElementById('fileInput');
            const uploadArea = document.getElementById('uploadArea');

            if (fileInput.files && fileInput.files[0]) {
                const file = fileInput.files[0];
                uploadArea.innerHTML = `✅ ${file.name}<br><small>Size: ${(file.size / 1024 / 1024).toFixed(2)}MB</small>`;
            }
        }

        async function startQuest() {
            const prompt = document.getElementById('promptInput').value.trim();
            if (!prompt) {
                alert('Please enter a prompt!');
                return;
            }

            if (currentMode === 'image') {
                const fileInput = document.getElementById('fileInput');
                if (!fileInput.files || !fileInput.files[0]) {
                    alert('Please upload an image!');
                    return;
                }
            }

            const questBtn = document.getElementById('questBtn');
            questBtn.disabled = true;
            questBtn.textContent = '🎨 Generating...';

            document.getElementById('resultsPanel').style.display = 'block';
            document.getElementById('statusMessage').innerHTML = '<div style="color: var(--success); padding: 15px; background: var(--bg-card); border-radius: 8px; margin-bottom: 20px;">🎨 Creating your masterpiece...</div>';
            document.getElementById('artworkResult').innerHTML = '';

            try {
                let response;

                if (currentMode === 'text') {
                    response = await fetch('/api/generate_text', {
                        method: 'POST',
                        headers: {
                            'Content-Type': 'application/json',
                            'X-API-KEY': API_KEY
                        },
                        body: JSON.stringify({
                            prompt: prompt,
                            negative_prompt: document.getElementById('negativePrompt').value.trim(),
                            style: selectedStyle,
                            num_inference_steps: parseInt(document.getElementById('stepsInput').value),
                            guidance_scale: parseFloat(document.getElementById('guidanceInput').value)
                        })
                    });
                } else {
                    const formData = new FormData();
                    formData.append('file', document.getElementById('fileInput').files[0]);
                    formData.append('prompt', prompt);
                    formData.append('negative_prompt', document.getElementById('negativePrompt').value.trim());
                    formData.append('style', selectedStyle);
                    formData.append('num_inference_steps', document.getElementById('stepsInput').value);
                    formData.append('guidance_scale', document.getElementById('guidanceInput').value);
                    formData.append('strength', document.getElementById('strengthInput').value);

                    response = await fetch('/api/generate_from_upload', {
                        method: 'POST',
                        headers: {
                            'X-API-KEY': API_KEY
                        },
                        body: formData
                    });
                }

                const result = await response.json();

                if (result.success) {
                    document.getElementById('statusMessage').innerHTML = '<div style="color: var(--success); padding: 15px; background: var(--bg-card); border-radius: 8px; margin-bottom: 20px;">🏆 Quest Complete! Masterpiece created!</div>';

                    let progressHTML = '';
                    if (result.player_progress) {
                        const progress = result.player_progress;
                        progressHTML = `
                            <div style="background: var(--bg-card); padding: 15px; border-radius: 8px; margin: 20px 0;">
                                <div style="color: var(--primary-gold); font-weight: bold;">+${progress.xp_earned} XP Earned!</div>
                                ${progress.level_up ? `<div style="color: var(--legendary-purple); font-weight: bold; margin-top: 5px;">🎉 LEVEL UP! You are now level ${progress.new_level}!</div>` : ''}
                                ${progress.new_achievements.length > 0 ? `<div style="color: var(--epic-orange); font-weight: bold; margin-top: 5px;">🏆 ${progress.new_achievements.length} Achievement(s) Unlocked!</div>` : ''}
                            </div>
                        `;
                    }

                    document.getElementById('artworkResult').innerHTML = `
                        ${progressHTML}
                        <img src="${result.base64}" class="generated-image" alt="Generated Artwork">
                        <div style="margin-top: 15px; font-size: 0.9rem; color: var(--text-secondary);">
                            Style: ${result.style_applied} (${result.style_tier}) |
                            Steps: ${result.metadata.steps} |
                            Guidance: ${result.metadata.guidance}
                        </div>
                    `;

                    // Refresh player data
                    await loadPlayerProfile();
                    updatePlayerHUD();
                    await loadStyles(); // Refresh to update unlocked styles

                } else {
                    document.getElementById('statusMessage').innerHTML = `<div style="color: var(--error); padding: 15px; background: var(--bg-card); border-radius: 8px; margin-bottom: 20px;">❌ Quest Failed: ${result.error}</div>`;
                }

            } catch (error) {
                console.error('Generation error:', error);
                document.getElementById('statusMessage').innerHTML = `<div style="color: var(--error); padding: 15px; background: var(--bg-card); border-radius: 8px; margin-bottom: 20px;">❌ Network Error: ${error.message}</div>`;
            } finally {
                questBtn.disabled = false;
                questBtn.textContent = '🚀 Begin Art Quest';
            }
        }
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return GAMIFIED_HTML

print("🎮 RWC-AI-Studio | The AI Art Quest - Gamified User interface configured !")

In [ ]:
1# 🚀 LAUNCH SERVER WITH NGROK
import threading
from pyngrok import ngrok

def run_flask_server():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask server in background thread
server_thread = threading.Thread(target=run_flask_server, daemon=True)
server_thread.start()

# Wait for server to start
import time
time.sleep(3)

# Create ngrok tunnel
try:
    public_url = ngrok.connect(5000)
    print("\n" + "="*60)
    print("🎮 RWC-AI-Studio | The AI Art Quest IS LIVE! 🎮")
    print("="*60)
    print(f"🌐 Public URL: {public_url}")
    print(f"🔑 API Key: {API_KEY}")
    print(f"🎯 Device: {DEVICE}")
    print(f"🤖 Models Loaded: Text2Img={text_pipe is not None}, Img2Img={pipe is not None}")
    print(f"🎨 Styles Available: {len(GAMIFIED_STYLE_PRESETS)}")
    print("\n🎮 HOW TO USE:")
    print(f"1. 🌐 Visit: {public_url}")
    print("2. 🎯 Choose your quest mode (Text or Transform)")
    print("3. 🎨 Select an art style (unlock more by leveling up!)")
    print("4. ✏️ Write your prompt and negative prompt")
    print("5. 🚀 Click 'Begin Art Quest' to generate!")
    print("6. 🏆 Earn XP, level up, and unlock legendary styles!")
    print("\n💡 PRO TIPS:")
    print("• Higher steps = better quality but slower generation")
    print("• Try different styles to unlock achievements")
    print("• Epic and Legendary styles give massive XP bonuses!")
    print("• Use detailed prompts for best results")
    print("="*60)

except Exception as e:
    print(f"❌ Failed to create ngrok tunnel: {e}")
    print("🌐 Server running locally at: http://localhost:5000")
    print(f"🔑 API Key: {API_KEY}")

In [ ]:
# 🎯 GENERATION EXAMPLES
print("🎮 ========== QUEST EXAMPLES ========== 🎮")
print("🎨 You can also generate directly in the notebook!")

print(f"\n👤 Your Current Stats:")
print(f"  🎯 Level: {CURRENT_PLAYER['player_level']}")
print(f"  ⭐ Total XP: {CURRENT_PLAYER['total_xp']}")
print(f"  🎨 Artworks Created: {CURRENT_PLAYER['total_generations']}")
print(f"  🏆 Achievements: {len(CURRENT_PLAYER['achievements_unlocked'])}")

print("\n🎨 Available Styles for Your Level:")
available_styles = []
for style_id, style_data in GAMIFIED_STYLE_PRESETS.items():
    if CURRENT_PLAYER["player_level"] >= style_data["unlock_level"]:
        available_styles.append(f"✅ {style_id} ({style_data['tier']} - +{style_data['xp_bonus']} XP)")
    else:
        available_styles.append(f"🔒 {style_id} (Level {style_data['unlock_level']} required)")

for style in available_styles[:10]:  # Show first 10
    print(f"  {style}")

if len(available_styles) > 10:
    print(f"  ... and {len(available_styles) - 10} more!")

print("\n🚀 Ready to generate! Use the web interface above or run the example below:")

In [ ]:
# QUICK GENERATION
print("🎯 ========== QUICK GENERATION ========== 🎯")
print("🎨 Generate directly in the notebook!")

# Customize these parameters
quest_prompt = "a mythical warrior, black tattered cloak, demonic mask, dual katanas glowing red, standing on a burning battlefield, ominous aura, moonlit night with red sky, cinematic anime scene, detailed shading."  # ✏️ Change this!
quest_style = "fantasy"  # ✏️ Try: fantasy, anime, cyberpunk, photorealistic
quest_negative = "blurry, low quality, dark"  # ✏️ What to avoid
quest_steps = 430  # ✏️ Higher = better quality (10-100)
quest_guidance = 7.5  # ✏️ Higher = follows prompt more (1-20)

print(f"🎨 Generating: {quest_prompt}")
print(f"🎭 Style: {quest_style}")
print(f"⚙️ Steps: {quest_steps}, Guidance: {quest_guidance}")

# Generate artwork
if text_pipe is not None:
    result = generate_art_quest(
        prompt=quest_prompt,
        style_id=quest_style,
        negative_prompt=quest_negative,
        steps=quest_steps,
        guidance=quest_guidance
    )

    if result["success"]:
        print(f"\n🏆 Quest Complete!")
        print(f"🎨 Style: {result['style_applied']} ({result['style_tier']} tier)")
        print(f"📊 Earned: {result['player_progress']['xp_earned']} XP")
        print(f"📁 Saved: {result['filename']}")

        # Display the generated image
        from IPython.display import display, HTML
        display(result["image"])

        if result['player_progress']['level_up']:
            print(f"\n🎉 LEVEL UP! You reached level {result['player_progress']['new_level']}!")

        if result['player_progress']['new_achievements']:
            print("\n🏅 New Achievements Unlocked:")
            for achievement in result['player_progress']['new_achievements']:
                print(f"  🏆 {achievement['name']}: {achievement['description']} (+{achievement['xp_reward']} XP)")

        print(f"\n📊 Updated Stats:")
        print(f"  🎯 Level: {CURRENT_PLAYER['player_level']}")
        print(f"  ⭐ Total XP: {CURRENT_PLAYER['total_xp']}")
        print(f"  🎨 Artworks: {CURRENT_PLAYER['total_generations']}")

    else:
        print(f"❌ Quest failed: {result['error']}")
        if "requires level" in result['error']:
            print("💡 Try using a style that matches your current level!")
else:
    print("❌ Text2Img pipeline not loaded. Please run the model loading cells first.")

print("\n💡 Want to try more? Edit the parameters above and run this cell again!")
print(f"🌐 Or use the full web interface: {public_url if 'public_url' in locals() else 'http://localhost:5000'}")

## 🎮 RWC-AI-Studio | The AI Art Quest - Complete!

🏆 **Congratulations! Your gamified AI art generation platform is running!**

### 🌐 **Web Interface Features:**
- **🎮 Full Gaming UI** - Complete RPG-style interface with progress tracking
- **🎨 Style Selection** - Visual style picker with tier filtering
- **📊 Real-time Stats** - Live XP, level, and achievement tracking
- **🖼️ Dual Modes** - Text-to-image and image-to-image generation
- **📱 Mobile Responsive** - Works perfectly on all devices

### 🎯 **Gaming Features:**
- **🏆 Level System** - Progress from Level 1 to 50
- **🎨 Tiered Styles** - Unlock Legendary styles as you level up
- **🏅 Achievements** - Complete challenges for bonus XP
- **💎 XP Rewards** - Higher tier styles give more experience
- **🎮 Progress Tracking** - Persistent player data and statistics

### 🚀 **What's New:**
- **✅ Fixed JSON Parsing** - Corrected all syntax errors
- **🌐 Ngrok Integration** - Automatic public URL generation
- **🔐 Secure Authentication** - API key protection for generation
- **⚡ Optimized Performance** - Fast 30-35 step generation
- **🎨 Enhanced Styles** - 20+ gamified art styles with descriptions

### 🎮 **How to Play:**
1. **🌐 Visit the Public URL** - Use the link generated above
2. **🎯 Choose Quest Mode** - Text Quest or Transform Quest
3. **🎨 Select Art Style** - Start with Common, unlock Legendary!
4. **✏️ Write Prompts** - Describe your artistic vision
5. **🚀 Generate Art** - Click "Begin Art Quest"
6. **🏆 Level Up** - Earn XP and unlock new styles!

---

**🎊 Happy Art Questing! Create amazing AI artwork and become a legendary artist! 🎊**